<a href="https://colab.research.google.com/github/dalanocau/MCD/blob/main/Proyecto_Final_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final - Proceso ETL
## Extract, Transform, Load - Sales Data Warehouse

Este notebook implementa un proceso ETL completo para construir un Data Warehouse de ventas.

## 0. Montar Google Drive y Configurar Acceso

In [ ]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

print("\n✓ Google Drive montado exitosamente")

Mounted at /content/drive

✓ Google Drive montado exitosamente


## 1. Configuración Inicial e Importaciones

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. EXTRACT - Carga de Datos desde Google Drive

In [ ]:
# ==============================
# DESCARGA DE ARCHIVOS DESDE GOOGLE DRIVE
# ==============================

import os
import gdown

# IDs de Google Drive (extraídos de los URLs)
catalog_id  = '1Ra91KNqXQ2yM0RA8Ddd4dM_HjXtDicn3'
web_id      = '1RZyQsNSHeoQ5JOQu_PT54etWPnCsVH_C'
products_id = '1DKpSR4W0A2IYALtxTPX6VhryHxNlXwFy'

# Nombres locales de los archivos
catalog_path  = 'Catalog_Orders.txt'
web_path      = 'Web_Orders.txt'
products_path = 'Products.txt'

# Descargar archivos
print("Descargando archivos...\n")
gdown.download(f'https://drive.google.com/uc?id={catalog_id}', catalog_path, quiet=False)
gdown.download(f'https://drive.google.com/uc?id={web_id}', web_path, quiet=False)
gdown.download(f'https://drive.google.com/uc?id={products_id}', products_path, quiet=False)

# Verificar existencia de archivos
print("\nVerificando archivos descargados...\n")
for nombre, path in [
    ('Catalog_Orders', catalog_path),
    ('Web_Orders', web_path),
    ('Products', products_path)
]:
    if os.path.exists(path):
        size = os.path.getsize(path) / 1024  # KB
        print(f"✓ {nombre}.txt - Encontrado ({size:.2f} KB)")
    else:
        print(f"✗ {nombre}.txt - NO ENCONTRADO")


Descargando archivos...



Downloading...
From: https://drive.google.com/uc?id=1Ra91KNqXQ2yM0RA8Ddd4dM_HjXtDicn3
To: /content/Catalog_Orders.txt
100%|██████████| 435k/435k [00:00<00:00, 62.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RZyQsNSHeoQ5JOQu_PT54etWPnCsVH_C
To: /content/Web_Orders.txt
100%|██████████| 73.3k/73.3k [00:00<00:00, 41.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DKpSR4W0A2IYALtxTPX6VhryHxNlXwFy
To: /content/Products.txt
100%|██████████| 13.0k/13.0k [00:00<00:00, 15.9MB/s]


Verificando archivos descargados...

✓ Catalog_Orders.txt - Encontrado (424.70 KB)
✓ Web_Orders.txt - Encontrado (71.55 KB)
✓ Products.txt - Encontrado (12.65 KB)


In [ ]:
# Extraer datos web (formato especial: header con comas, datos con punto y coma)
print("Cargando datos web...")

with open(web_path, encoding='latin1') as f:
    lines = f.readlines()

# Encabezado (primera línea, separado por comas)
header = lines[0].replace('"', '').strip().split(',')

# Datos (resto de líneas, separadas por punto y coma)
data = [line.strip().replace('"', '').split(';') for line in lines[1:]]

# Crear DataFrame
web_df = pd.DataFrame(data, columns=header)

print(f"✓ Datos web cargados: {len(web_df)} registros")
print("\nPrimeras filas:")
display(web_df.head())

Cargando datos web...
✓ Datos web cargados: 943 registros

Primeras filas:


,ID,INV,DATE,CATALOG,PCODE,QTY,custnum
0,1,2513000.00,GD3200,17/12/2000 00:00:00,Gardening,3.00,"Rifkin, Mr. Bob"
1,2,2513000.00,PT1400,6/1/2001 00:00:00,Pets,1.00,"Rifkin, Mr. Bob"
2,3,2513000.00,PT2000,6/1/2001 00:00:00,Pet,1.00,"Rifkin, Mr. Bob"
3,4,2513000.00,PT2100,6/1/2001 00:00:00,Pets,1.00,"Rifkin, Mr. Bob"
4,5,2513000.00,PT2OOO,27/7/2000 00:00:00,Pets,4.00,"Rifkin, Mr. Bob"


In [ ]:
# Extraer datos del catálogo
print("Cargando catálogo...")

catalog_df = pd.read_csv(
    catalog_path,
    sep=',',
    engine='python',
    quotechar='"',
    on_bad_lines='skip',
    encoding='latin1'
)

print(f"✓ Catálogo cargado: {len(catalog_df)} registros")
print("\nPrimeras filas:")
display(catalog_df.head())

Cargando catálogo...
✓ Catálogo cargado: 6766 registros

Primeras filas:


,ID,INV,DATE,CATALOG,PCODE,QTY,custnum
0,1,107707.00,3/97/7 00:00:00,Sports,SP1000,1.00,1242
1,2,110633.00,3/97/25 00:00:00,Gardening,GD2200,1.00,1243
2,3,111155.00,3/97/28 00:00:00,Pets,PT1300,3.00,1244
3,4,126867.00,6/97/22 00:00:00,Toys,TY4100,1.00,1245
4,5,135872.00,8/97/8 00:00:00,Toys,TY1200,4.00,1246


In [ ]:
# Extraer datos de productos
print("Cargando productos...")

products_df = pd.read_csv(
    products_path,
    sep=',',
    encoding='latin1',
    on_bad_lines='skip'
)

print(f"✓ Productos cargados: {len(products_df)} registros")
print("\nPrimeras filas:")
display(products_df.head())

Cargando productos...
✓ Productos cargados: 192 registros

Primeras filas:


,ID,TYPE,DESCRIP,PRICE,COST,PCODE,supplier
0,1,Database,Track-It-All,560.00,425.00,SW1005,"Software America, Inc."
1,2,Database,Omnibus,499.95,450.00,SW1207,Software America
2,3,Database,Balboa,499.00,300.00,SW3049,Software America
3,4,Financial,Stock Market Analyst,69.95,45.00,SW2842,Software America
4,5,Financial,Tax Relief (Version 10.40),77.95,54.50,SW4187,Software America


## 3. TRANSFORM - Limpieza y Transformación de Datos

### 3.1 Exploración y Calidad de Datos

In [ ]:
# Verificar estructura de datos
print("=" * 50)
print("DATOS WEB")
print("=" * 50)
print(f"Dimensiones: {web_df.shape}")
print(f"\nColumnas: {web_df.columns.tolist()}")
print(f"\nTipos de datos:\n{web_df.dtypes}")
print(f"\nValores nulos:\n{web_df.isnull().sum()}")

print("\n" + "=" * 50)
print("CATÁLOGO")
print("=" * 50)
print(f"Dimensiones: {catalog_df.shape}")
print(f"\nColumnas: {catalog_df.columns.tolist()}")
print(f"\nTipos de datos:\n{catalog_df.dtypes}")
print(f"\nValores nulos:\n{catalog_df.isnull().sum()}")

print("\n" + "=" * 50)
print("PRODUCTOS")
print("=" * 50)
print(f"Dimensiones: {products_df.shape}")
print(f"\nColumnas: {products_df.columns.tolist()}")
print(f"\nTipos de datos:\n{products_df.dtypes}")
print(f"\nValores nulos:\n{products_df.isnull().sum()}")

DATOS WEB
Dimensiones: (943, 7)

Columnas: ['ID', 'INV', 'DATE', 'CATALOG', 'PCODE', 'QTY', 'custnum']

Tipos de datos:
ID         object
INV        object
DATE       object
CATALOG    object
PCODE      object
QTY        object
custnum    object
dtype: object

Valores nulos:
ID         0
INV        0
DATE       0
CATALOG    0
PCODE      0
QTY        0
custnum    0
dtype: int64

CATÁLOGO
Dimensiones: (6766, 7)

Columnas: ['ID', 'INV', 'DATE', 'CATALOG', 'PCODE', 'QTY', 'custnum']

Tipos de datos:
ID           int64
INV        float64
DATE        object
CATALOG     object
PCODE       object
QTY         object
custnum      int64
dtype: object

Valores nulos:
ID         0
INV        0
DATE       0
CATALOG    2
PCODE      0
QTY        6
custnum    0
dtype: int64

PRODUCTOS
Dimensiones: (192, 7)

Columnas: ['ID', 'TYPE', 'DESCRIP', 'PRICE', 'COST', 'PCODE', 'supplier']

Tipos de datos:
ID            int64
TYPE         object
DESCRIP      object
PRICE       float64
COST        float64
PCODE  

### 3.2 Limpieza de Datos Web

In [ ]:
# Limpiar y transformar datos web
print("Limpiando datos web...")

# Copiar para no modificar el original
web_clean = web_df.copy()

# Convertir tipos de datos
web_clean['ID'] = pd.to_numeric(web_clean['ID'], errors='coerce')
web_clean['INV'] = pd.to_numeric(web_clean['INV'], errors='coerce')
web_clean['QTY'] = pd.to_numeric(web_clean['QTY'], errors='coerce')

# Limpiar columna PCODE (códigos de producto)
web_clean['PCODE'] = web_clean['PCODE'].str.strip()

# Estandarizar columna CATALOG
web_clean['CATALOG'] = web_clean['CATALOG'].str.strip().str.title()

print(f"✓ Registros antes de limpieza: {len(web_df)}")
print(f"✓ Registros después de limpieza: {len(web_clean)}")
print(f"\nPrimeras filas limpias:")
display(web_clean.head())

Limpiando datos web...
✓ Registros antes de limpieza: 943
✓ Registros después de limpieza: 943

Primeras filas limpias:


,ID,INV,DATE,CATALOG,PCODE,QTY,custnum
0,1,2513000.00,GD3200,17/12/2000 00:00:00,Gardening,3.00,"Rifkin, Mr. Bob"
1,2,2513000.00,PT1400,6/1/2001 00:00:00,Pets,1.00,"Rifkin, Mr. Bob"
2,3,2513000.00,PT2000,6/1/2001 00:00:00,Pet,1.00,"Rifkin, Mr. Bob"
3,4,2513000.00,PT2100,6/1/2001 00:00:00,Pets,1.00,"Rifkin, Mr. Bob"
4,5,2513000.00,PT2OOO,27/7/2000 00:00:00,Pets,4.00,"Rifkin, Mr. Bob"


### 3.3 Limpieza de Catálogo

In [ ]:
# Limpiar y transformar catálogo
print("Limpiando catálogo...")

catalog_clean = catalog_df.copy()

# Convertir tipos de datos
catalog_clean['ID'] = pd.to_numeric(catalog_clean['ID'], errors='coerce')
catalog_clean['INV'] = pd.to_numeric(catalog_clean['INV'], errors='coerce')
catalog_clean['QTY'] = pd.to_numeric(catalog_clean['QTY'], errors='coerce')
catalog_clean['custnum'] = pd.to_numeric(catalog_clean['custnum'], errors='coerce')

# Limpiar PCODE
catalog_clean['PCODE'] = catalog_clean['PCODE'].str.strip()

# Estandarizar CATALOG
catalog_clean['CATALOG'] = catalog_clean['CATALOG'].str.strip().str.title()

print(f"✓ Registros limpios: {len(catalog_clean)}")
display(catalog_clean.head())

Limpiando catálogo...
✓ Registros limpios: 6766


,ID,INV,DATE,CATALOG,PCODE,QTY,custnum
0,1,107707.00,3/97/7 00:00:00,Sports,SP1000,1.00,1242
1,2,110633.00,3/97/25 00:00:00,Gardening,GD2200,1.00,1243
2,3,111155.00,3/97/28 00:00:00,Pets,PT1300,3.00,1244
3,4,126867.00,6/97/22 00:00:00,Toys,TY4100,1.00,1245
4,5,135872.00,8/97/8 00:00:00,Toys,TY1200,4.00,1246


### 3.4 Limpieza de Productos

In [ ]:
# Limpiar y transformar productos
print("Limpiando productos...")

products_clean = products_df.copy()

# Convertir tipos de datos numéricos
numeric_cols = ['ID', 'PRICE', 'COST']
for col in numeric_cols:
    if col in products_clean.columns:
        products_clean[col] = pd.to_numeric(products_clean[col], errors='coerce')

# Limpiar PCODE
products_clean['PCODE'] = products_clean['PCODE'].str.strip()

# Estandarizar TYPE
if 'TYPE' in products_clean.columns:
    products_clean['TYPE'] = products_clean['TYPE'].str.strip().str.title()

print(f"✓ Registros limpios: {len(products_clean)}")
display(products_clean.head())

Limpiando productos...
✓ Registros limpios: 192


,ID,TYPE,DESCRIP,PRICE,COST,PCODE,supplier
0,1,Database,Track-It-All,560.00,425.00,SW1005,"Software America, Inc."
1,2,Database,Omnibus,499.95,450.00,SW1207,Software America
2,3,Database,Balboa,499.00,300.00,SW3049,Software America
3,4,Financial,Stock Market Analyst,69.95,45.00,SW2842,Software America
4,5,Financial,Tax Relief (Version 10.40),77.95,54.50,SW4187,Software America


### 3.5 Procesamiento de Fechas

In [ ]:
# Función para procesar fechas
def parse_date(date_str):
    """
    Convierte varios formatos de fecha a formato estándar YYYYMMDD
    """
    try:
        # Eliminar espacios y hora
        date_str = str(date_str).split()[0]

        # Intentar diferentes formatos
        for fmt in ['%d/%m/%Y', '%m/%d/%Y', '%Y/%m/%d', '%d-%m-%Y', '%Y-%m-%d']:
            try:
                dt = pd.to_datetime(date_str, format=fmt)
                return int(dt.strftime('%Y%m%d'))
            except:
                continue

        # Si no funciona ninguno, intentar parseo automático
        dt = pd.to_datetime(date_str)
        return int(dt.strftime('%Y%m%d'))
    except:
        return None

# Aplicar a web_clean
if 'DATE' in web_clean.columns:
    print("Procesando fechas en datos web...")
    web_clean['date_key'] = web_clean['DATE'].apply(parse_date)
    print(f"✓ Fechas procesadas: {web_clean['date_key'].notna().sum()} de {len(web_clean)}")

# Aplicar a catalog_clean
if 'DATE' in catalog_clean.columns:
    print("\nProcesando fechas en catálogo...")
    catalog_clean['date_key'] = catalog_clean['DATE'].apply(parse_date)
    print(f"✓ Fechas procesadas: {catalog_clean['date_key'].notna().sum()} de {len(catalog_clean)}")

Procesando fechas en datos web...
✓ Fechas procesadas: 0 de 943

Procesando fechas en catálogo...
✓ Fechas procesadas: 818 de 6766


### 3.6 Integración de Datos

In [ ]:
# Combinar datos de diferentes fuentes
print("Integrando datos...")

# Unir web y catalog
all_sales = pd.concat([web_clean, catalog_clean], ignore_index=True)
print(f"✓ Total de ventas combinadas: {len(all_sales)}")

# Unir con productos para obtener información adicional
sales_with_products = all_sales.merge(
    products_clean[['PCODE', 'PRICE', 'COST', 'TYPE', 'DESCRIP']],
    on='PCODE',
    how='left'
)

print(f"✓ Ventas con información de productos: {len(sales_with_products)}")
print(f"\nColumnas finales: {sales_with_products.columns.tolist()}")
display(sales_with_products.head())

Integrando datos...
✓ Total de ventas combinadas: 7709
✓ Ventas con información de productos: 7709

Columnas finales: ['ID', 'INV', 'DATE', 'CATALOG', 'PCODE', 'QTY', 'custnum', 'date_key', 'PRICE', 'COST', 'TYPE', 'DESCRIP']


/tmp/ipython-input-3645067415.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_sales = pd.concat([web_clean, catalog_clean], ignore_index=True)


,ID,INV,DATE,CATALOG,PCODE,QTY,custnum,date_key,PRICE,COST,TYPE,DESCRIP
0,1,2513000.00,GD3200,17/12/2000 00:00:00,Gardening,3.00,"Rifkin, Mr. Bob",NaN,NaN,NaN,NaN,NaN
1,2,2513000.00,PT1400,6/1/2001 00:00:00,Pets,1.00,"Rifkin, Mr. Bob",NaN,NaN,NaN,NaN,NaN
2,3,2513000.00,PT2000,6/1/2001 00:00:00,Pet,1.00,"Rifkin, Mr. Bob",NaN,NaN,NaN,NaN,NaN
3,4,2513000.00,PT2100,6/1/2001 00:00:00,Pets,1.00,"Rifkin, Mr. Bob",NaN,NaN,NaN,NaN,NaN
4,5,2513000.00,PT2OOO,27/7/2000 00:00:00,Pets,4.00,"Rifkin, Mr. Bob",NaN,NaN,NaN,NaN,NaN


### 3.7 Creación de Dimensiones y Tabla de Hechos

In [ ]:
# Crear dimensión de productos
print("Creando dimensión de productos...")

dim_product = products_clean.copy()
dim_product['product_key'] = dim_product['ID']

# Seleccionar columnas relevantes
dim_product = dim_product[[
    'product_key', 'PCODE', 'TYPE', 'DESCRIP', 'PRICE', 'COST'
]].drop_duplicates(subset=['PCODE'])

print(f"✓ Productos únicos: {len(dim_product)}")
display(dim_product.head())

Creando dimensión de productos...
✓ Productos únicos: 192


,product_key,PCODE,TYPE,DESCRIP,PRICE,COST
0,1,SW1005,Database,Track-It-All,560.00,425.00
1,2,SW1207,Database,Omnibus,499.95,450.00
2,3,SW3049,Database,Balboa,499.00,300.00
3,4,SW2842,Financial,Stock Market Analyst,69.95,45.00
4,5,SW4187,Financial,Tax Relief (Version 10.40),77.95,54.50


In [ ]:
# Crear dimensión de fechas
print("Creando dimensión de fechas...")

# Obtener todas las fechas únicas
all_dates = pd.concat([
    sales_with_products['date_key'].dropna()
]).unique()

dim_date = pd.DataFrame({'date_key': sorted(all_dates)})

# Agregar atributos de fecha
dim_date['date_key'] = dim_date['date_key'].astype(int)
dim_date['full_date'] = pd.to_datetime(dim_date['date_key'], format='%Y%m%d')
dim_date['year'] = dim_date['full_date'].dt.year
dim_date['month'] = dim_date['full_date'].dt.month
dim_date['day'] = dim_date['full_date'].dt.day
dim_date['quarter'] = dim_date['full_date'].dt.quarter
dim_date['day_of_week'] = dim_date['full_date'].dt.dayofweek
dim_date['month_name'] = dim_date['full_date'].dt.month_name()
dim_date['day_name'] = dim_date['full_date'].dt.day_name()

print(f"✓ Fechas únicas: {len(dim_date)}")
display(dim_date.head())

Creando dimensión de fechas...
✓ Fechas únicas: 132


,date_key,full_date,year,month,day,quarter,day_of_week,month_name,day_name
0,20010101,2001-01-01,2001,1,1,1,0,January,Monday
1,20010201,2001-02-01,2001,2,1,1,3,February,Thursday
2,20010301,2001-03-01,2001,3,1,1,3,March,Thursday
3,20010401,2001-04-01,2001,4,1,2,6,April,Sunday
4,20010501,2001-05-01,2001,5,1,2,1,May,Tuesday


In [ ]:
# Crear tabla de hechos de ventas
print("Creando tabla de hechos de ventas...")

# Mapear PCODE a product_key
pcode_to_key = dict(zip(dim_product['PCODE'], dim_product['product_key']))
sales_with_products['product_key'] = sales_with_products['PCODE'].map(pcode_to_key)

# Crear fact table
fact_sales = sales_with_products[[
    'product_key', 'date_key', 'QTY', 'PRICE'
]].copy()

# Renombrar columnas
fact_sales.columns = ['product_key', 'date_key', 'quantity', 'unit_price']

# Calcular monto total
fact_sales['total_amount'] = fact_sales['quantity'] * fact_sales['unit_price']

# Eliminar nulos
fact_sales = fact_sales.dropna(subset=['product_key', 'date_key'])

print(f"✓ Registros en fact_sales (antes de agregación): {len(fact_sales)}")
display(fact_sales.head())

Creando tabla de hechos de ventas...
✓ Registros en fact_sales (antes de agregación): 793


,product_key,date_key,quantity,unit_price,total_amount
994,121.00,20220501.00,1.00,12.00,12.00
1037,111.00,20260201.00,3.00,8.00,24.00
1038,179.00,20210301.00,1.00,79.99,79.99
1039,115.00,20080401.00,2.00,12.00,24.00
1059,121.00,20190201.00,1.00,12.00,12.00


### 3.8 Gestión de Duplicados y Agregación

In [ ]:
# Verificar duplicados en la tabla de hechos
duplicates = fact_sales.duplicated(subset=['product_key', 'date_key']).sum()
print(f"Duplicados encontrados (product_key, date_key): {duplicates}")

if duplicates > 0:
    print("\nEjemplos de duplicados:")
    duplicate_rows = fact_sales[fact_sales.duplicated(subset=['product_key', 'date_key'], keep=False)]
    display(duplicate_rows.sort_values(['product_key', 'date_key']).head(10))

Duplicados encontrados (product_key, date_key): 362

Ejemplos de duplicados:


,product_key,date_key,quantity,unit_price,total_amount
3288,2.00,20010101.00,1.00,499.95,499.95
6292,2.00,20010101.00,1.00,499.95,499.95
1303,5.00,20170501.00,1.00,77.95,77.95
4307,5.00,20170501.00,1.00,77.95,77.95
3364,5.00,20300301.00,1.00,77.95,77.95
6368,5.00,20300301.00,1.00,77.95,77.95
2393,13.00,20190101.00,3.00,36.50,109.50
5397,13.00,20190101.00,3.00,36.50,109.50
2712,13.00,20280301.00,1.00,36.50,36.50
5716,13.00,20280301.00,1.00,36.50,36.50


In [ ]:
# Agregar ventas por producto y fecha
print("Agregando ventas por producto y fecha...")

fact_sales_agg = (
    fact_sales
    .groupby(['product_key', 'date_key'], as_index=False)
    .agg({
        'quantity': 'sum',
        'total_amount': 'sum'
    })
)

# Convertir tipos de datos
fact_sales_agg['product_key'] = fact_sales_agg['product_key'].astype(int)
fact_sales_agg['date_key'] = fact_sales_agg['date_key'].astype(int)

print(f"\n✓ Registros antes de agregación: {len(fact_sales)}")
print(f"✓ Registros después de agregación: {len(fact_sales_agg)}")
print(f"✓ Duplicados eliminados: {len(fact_sales) - len(fact_sales_agg)}")

# Verificar que no hay duplicados
final_duplicates = fact_sales_agg.duplicated(subset=['product_key', 'date_key']).sum()
print(f"\n✓ Duplicados finales: {final_duplicates}")

display(fact_sales_agg.head())

Agregando ventas por producto y fecha...

✓ Registros antes de agregación: 793
✓ Registros después de agregación: 431
✓ Duplicados eliminados: 362

✓ Duplicados finales: 0


,product_key,date_key,quantity,total_amount
0,2,20010101,2.00,999.90
1,5,20170501,2.00,155.90
2,5,20300301,2.00,155.90
3,9,20010201,1.00,35.50
4,10,20110101,3.00,149.85


### 3.9 Validación de Calidad de Datos

In [ ]:
# Validación final de datos
print("=" * 50)
print("VALIDACIÓN DE CALIDAD DE DATOS")
print("=" * 50)

print("\n1. FACT_SALES")
print("-" * 50)
print(f"Total de registros: {len(fact_sales_agg)}")
print(f"Columnas: {fact_sales_agg.columns.tolist()}")
print(f"Tipos de datos:\n{fact_sales_agg.dtypes}")
print(f"\nValores nulos:\n{fact_sales_agg.isnull().sum()}")
print(f"\nEstadísticas:")
display(fact_sales_agg[['quantity', 'total_amount']].describe())
print(f"\nTotales:")
print(f"  Cantidad total: {fact_sales_agg['quantity'].sum():,.0f}")
print(f"  Monto total: ${fact_sales_agg['total_amount'].sum():,.2f}")

print("\n2. DIM_PRODUCT")
print("-" * 50)
print(f"Total de productos: {len(dim_product)}")
print(f"Productos únicos: {dim_product['product_key'].nunique()}")

print("\n3. DIM_DATE")
print("-" * 50)
print(f"Total de fechas: {len(dim_date)}")
print(f"Rango de fechas: {dim_date['full_date'].min()} a {dim_date['full_date'].max()}")
print(f"Años: {sorted(dim_date['year'].unique())}")

VALIDACIÓN DE CALIDAD DE DATOS

1. FACT_SALES
--------------------------------------------------
Total de registros: 431
Columnas: ['product_key', 'date_key', 'quantity', 'total_amount']
Tipos de datos:
product_key       int64
date_key          int64
quantity        float64
total_amount    float64
dtype: object

Valores nulos:
product_key     0
date_key        0
quantity        0
total_amount    0
dtype: int64

Estadísticas:


,quantity,total_amount
count,431.00,431.00
mean,2.97,176.88
std,2.01,310.85
min,0.00,0.00
25%,2.00,25.99
50%,2.00,63.00
75%,4.00,173.25
max,10.00,2880.00



Totales:
  Cantidad total: 1,278
  Monto total: $76,235.96

2. DIM_PRODUCT
--------------------------------------------------
Total de productos: 192
Productos únicos: 192

3. DIM_DATE
--------------------------------------------------
Total de fechas: 132
Rango de fechas: 2001-01-01 00:00:00 a 2031-05-01 00:00:00
Años: [np.int32(2001), np.int32(2002), np.int32(2003), np.int32(2004), np.int32(2005), np.int32(2006), np.int32(2007), np.int32(2008), np.int32(2009), np.int32(2010), np.int32(2011), np.int32(2012), np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024), np.int32(2025), np.int32(2026), np.int32(2027), np.int32(2028), np.int32(2029), np.int32(2030), np.int32(2031)]


## 4. LOAD - Carga de Datos al Data Warehouse (Google Drive)

In [ ]:
# Definir directorio de salida en Google Drive
output_dir = '/content/drive/MyDrive/data_warehouse_output'
os.makedirs(output_dir, exist_ok=True)

print(f"Guardando datos en Google Drive: {output_dir}")
print("=" * 50)

# Guardar tabla de hechos
fact_sales_path = os.path.join(output_dir, 'fact_sales_dw.csv')
fact_sales_agg.to_csv(fact_sales_path, index=False, encoding='utf-8')
print(f"✓ Tabla de hechos guardada: fact_sales_dw.csv")
print(f"  Registros: {len(fact_sales_agg)}")

# Guardar dimensión de productos
dim_product_path = os.path.join(output_dir, 'dim_product_dw.csv')
dim_product.to_csv(dim_product_path, index=False, encoding='utf-8')
print(f"\n✓ Dimensión de productos guardada: dim_product_dw.csv")
print(f"  Registros: {len(dim_product)}")

# Guardar dimensión de fechas
dim_date_path = os.path.join(output_dir, 'dim_date_dw.csv')
dim_date.to_csv(dim_date_path, index=False, encoding='utf-8')
print(f"\n✓ Dimensión de fechas guardada: dim_date_dw.csv")
print(f"  Registros: {len(dim_date)}")

print("\n" + "=" * 50)
print("PROCESO ETL COMPLETADO EXITOSAMENTE")
print("=" * 50)
print(f"\n📁 Los archivos están disponibles en:")
print(f"   {output_dir}")
print("\nPuedes acceder a ellos desde Google Drive en tu navegador.")

Guardando datos en Google Drive: /content/drive/MyDrive/data_warehouse_output
✓ Tabla de hechos guardada: fact_sales_dw.csv
  Registros: 431

✓ Dimensión de productos guardada: dim_product_dw.csv
  Registros: 192

✓ Dimensión de fechas guardada: dim_date_dw.csv
  Registros: 132

PROCESO ETL COMPLETADO EXITOSAMENTE

📁 Los archivos están disponibles en:
   /content/drive/MyDrive/data_warehouse_output

Puedes acceder a ellos desde Google Drive en tu navegador.


## 5. Verificación Final

In [ ]:
# Verificar archivos guardados
print("Verificando archivos guardados...\n")

# Leer fact_sales guardado
fact_verify = pd.read_csv(fact_sales_path)
print("FACT_SALES verificado:")
display(fact_verify.head())
print(f"\nRegistros: {len(fact_verify)}")
print(f"Duplicados: {fact_verify.duplicated(subset=['product_key', 'date_key']).sum()}")

# Estadísticas finales
print("\n" + "=" * 50)
print("RESUMEN FINAL DEL DATA WAREHOUSE")
print("=" * 50)
print(f"Total de ventas: {len(fact_verify):,}")
print(f"Total de productos: {len(dim_product):,}")
print(f"Total de fechas: {len(dim_date):,}")
print(f"\nCantidad total vendida: {fact_verify['quantity'].sum():,.0f} unidades")
print(f"Monto total de ventas: ${fact_verify['total_amount'].sum():,.2f}")

Verificando archivos guardados...

FACT_SALES verificado:


,product_key,date_key,quantity,total_amount
0,2,20010101,2.00,999.90
1,5,20170501,2.00,155.90
2,5,20300301,2.00,155.90
3,9,20010201,1.00,35.50
4,10,20110101,3.00,149.85



Registros: 431
Duplicados: 0

RESUMEN FINAL DEL DATA WAREHOUSE
Total de ventas: 431
Total de productos: 192
Total de fechas: 132

Cantidad total vendida: 1,278 unidades
Monto total de ventas: $76,235.96


## 6. Descargar archivos localmente

In [ ]:
# Si quieres descargar los archivos CSV directamente a tu computadora
from google.colab import files

print("Descargando archivos...")

files.download(fact_sales_path)
print("✓ fact_sales_dw.csv descargado")

files.download(dim_product_path)
print("✓ dim_product_dw.csv descargado")

files.download(dim_date_path)
print("✓ dim_date_dw.csv descargado")

print("\n¡Todos los archivos han sido descargados!")

Descargando archivos...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ fact_sales_dw.csv descargado


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ dim_product_dw.csv descargado


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ dim_date_dw.csv descargado

¡Todos los archivos han sido descargados!
